In [10]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import gmaps
import os
import scipy.stats as st
from scipy.stats import linregress


# Import keys
from config import w_api_key
from config import gkey

# Import citipy module
# Be sure to be in PythonData where this installed to have this work
from citipy import citipy

In [8]:
weather_data = pd.read_csv("../WeatherPy/City_Weather_data.csv")
weather_data

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.1500,-70.9167,68.00,52,0,11.50,CL,1612819286
1,bredasdorp,-34.5322,20.0403,62.60,88,5,5.75,ZA,1612819481
2,cidreira,-30.1811,-50.2056,74.77,71,0,12.21,BR,1612819184
3,mataura,-46.1927,168.8643,60.39,85,100,2.33,NZ,1612819485
4,ponta do sol,32.6667,-17.1000,57.99,69,24,3.00,PT,1612819390
...,...,...,...,...,...,...,...,...,...
505,camalu,30.8500,-116.0667,64.00,43,0,5.99,MX,1612820091
506,praia da vitoria,38.7333,-27.0667,60.80,77,40,13.00,PT,1612819766
507,kihei,20.7850,-156.4656,77.00,61,40,19.57,US,1612820093
508,constantine,36.3650,6.6147,50.00,76,75,20.71,DZ,1612820094


In [11]:
# Configure google heatmap
gmaps.configure(api_key=gkey)

In [12]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Long"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_data["Humidity"].astype(float)

In [13]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

# Dimensions seem a bit weird, but basic function seems to be working

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# What are my preferred vacations conditions?
# 60-80 degrees
# Average cloudiness, 45-70??
# Humidty 50-65%

In [44]:
# Narrow down the dataframe to ideal conditions
ideal_weather = weather_data.loc[weather_data["Humidity"] >= 30]
ideal_weather = ideal_weather.loc[weather_data["Humidity"] <= 80]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] >= 55]
ideal_weather = ideal_weather.loc[weather_data["Max Temp"] <= 85]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] >= 45]
ideal_weather = ideal_weather.loc[weather_data["Cloudiness"] <= 75]

ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,nago,26.5881,127.9761,57.20,72,75,10.36,JP,1612819497
21,puerto ayora,-0.7393,-90.3518,82.40,78,75,5.75,EC,1612819389
39,atuona,-9.8000,-139.0333,79.68,76,75,21.41,PF,1612819315
95,alice,27.7523,-98.0697,80.60,61,75,11.50,US,1612819595
130,pergamino,-33.8900,-60.5736,73.00,75,75,4.00,AR,1612819404
148,hobyo,5.3505,48.5268,77.09,74,75,21.99,SO,1612819659
160,cayenne,4.9333,-52.3333,82.40,69,75,12.66,GF,1612819673
206,nishihara,26.1842,127.7558,60.80,63,75,10.36,JP,1612819727
216,talcahuano,-36.7167,-73.1167,64.40,77,75,12.66,CL,1612819740
226,huaral,-11.4950,-77.2078,75.20,73,75,13.80,PE,1612819569


In [50]:
# Let's check we're talking to google

waycross = "31.2136, -82.3540"
search_type = "hotel"
radius = 5000


params = {
    "location": waycross,
    "radius": radius,
    "types": search_type,
    "key": gkey
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params)

# Ok! Well, it is basically working.  So that's a good sign

In [ ]:
# Find the first hotel within 5000 meters of each city
